# Detección de anomalías para identificar transacciones fraudulentas con tarjetas de crédito

---
### Integrantes:
* Juan Fernando Gallego
* Juan Esteban Chavarría
* Karla Orozco
* Jonathan Stiven Zapata


## <h1 style="text-align: center">1. Introducción</h1><a id = "1" ></a>

Cuando hacemos cualquier transacción al comprar productos en línea una buena cantidad de personas prefieren las tarjetas de crédito. El límite de crédito de las tarjetas de crédito a veces nos ayuda a hacer compras incluso si no tenemos la cantidad en ese momento. Pero, por otro lado, estas características son mal utilizadas por los atacantes cibernéticos.

Cuando hablamos de seguridad en la vida digital, el principal reto es encontrar la actividad anormal.

Anomalía es un sinónimo de la palabra "outlier". La detección de anomalías (o detección de valores atípicos) es la identificación de elementos, eventos u observaciones poco comunes que levantan sospechas por diferir significativamente de la mayoría de los datos. Las actividades anómalas pueden estar relacionadas con algún tipo de problema o suceso raro, como fraudes bancarios, problemas médicos, defectos estructurales, equipos que funcionan mal, etc.

Para hacer frente a este problema necesitamos un sistema que pueda abortar la transacción si la encuentra sospechosa. Aquí viene la necesidad de un sistema que pueda rastrear el patrón de todas las transacciones y si cualquier patrón es anormal entonces la transacción debe ser abortada.

Hoy en día, tenemos muchos algoritmos de aprendizaje automático que pueden ayudarnos a clasificar las transacciones anormales. El único requisito son los datos pasados y el algoritmo adecuado que pueda ajustarse a nuestros datos de la mejor manera.

Existen diversos métodos de aprendizaje tanto en el campo supervisado como no supervisado. Aquí, sin embargo, vamos a discutir cómo el aprendizaje no supervisado se utiliza para para encontrar outliers y también entender por qué la detección de anomalías utilizando el aprendizaje no supervisado es beneficioso en la mayoría de los casos.

En este ejercicio se planteará la solución al problema en cuestión mediante la aplicación de dos algoritmos de clasificación no supervisada los cuales serán comparados con el fin de decidir cual de estos representa una solución mas adecuada el problema planteado.

Finalmente, se obtendrá el mejor modelo que pueda clasificar la transacción en tipos normales y anormales.

// En este ejercicio se trabajará con el algoritmo K-means como algoritmo de clasificación no supervisada convencional y una variación del algoritmo utilizando DTW (Dynamic Time Warping).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<h1 style="text-align: center">2. Dataset</h1><a id = "1" ></a>

The dataset contains transactions made by credit cards in September 2013 by European cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.

In [ ]:
df = pd.read_csv("../input/loan-default-dataset/Loan_Default.csv")
df.head()

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         148670 non-null  int64  
 1   year                       148670 non-null  int64  
 2   loan_limit                 145326 non-null  object 
 3   Gender                     148670 non-null  object 
 4   approv_in_adv              147762 non-null  object 
 5   loan_type                  148670 non-null  object 
 6   loan_purpose               148536 non-null  object 
 7   Credit_Worthiness          148670 non-null  object 
 8   open_credit                148670 non-null  object 
 9   business_or_commercial     148670 non-null  object 
 10  loan_amount                148670 non-null  int64  
 11  rate_of_interest           112231 non-null  float64
 12  Interest_rate_spread       112031 non-null  float64
 13  Upfront_charges            10


We all know the Importance of good features for machine learning models. In Machine learning task we have features which we need to process to make them good and this is done by data preprocessing  tasks.

**Data Preprocessing** : Data preprocessing is a process of preparing the raw data and making it suitable for a machine learning model. It is the first and crucial step while creating a machine learning model. When creating a machine learning project, it is not always a case that we come across the clean and formatted data. And while doing any operation with data, it is mandatory to clean it and put in a formatted way. So for this, we use data preprocessing task.

A real-world data generally contains noises, missing values, and maybe in an unusable format which cannot be directly used for machine learning models. Data preprocessing is required tasks for cleaning the data and making it suitable for a machine learning model which also increases the accuracy and efficiency of a machine learning model.

Data Preprocessing involves below steps:

- Getting the dataset
- Importing libraries
- Importing datasets
- Finding Missing Data
- Encoding Categorical Data

Many of us know traditional approaches for above listed steps but in this notebook I will discuss some different approaches which could be game changer for your next project. 



Evaluating an Anomaly Detection Algorithm
confusion matrix

Drawback of This Anomaly Detection Algorithm


Analysing the Performance